In [0]:
# # content/aclImdb_v1.tar

# # content/sample_data/aclImdb_v1.tar

# !apt-get install p7zip-full
# !p7zip -d sample_data/aclImdb_v1.tar.7z
# !tar -xvf sample_data/aclImdb_v1.tar.tar


# !pip install pyunpack
# !pip install patool

from pyunpack import Archive
Archive('aclImdb_v1.tar').extractall('sample_data')
!tar -xvf aclImdb_v1.tar



In [0]:
import os

imdb_dir = 'aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [120]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # We will cut reviews after 100 words
training_samples = 200  # We will be training on 200 samples
validation_samples = 10000  # We will be validating on 10000 samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Found 88582 unique tokens.
Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils


In [0]:
encoder = LabelEncoder()
encoder.fit(y_train)
y_trains = encoder.transform(y_train)
y_tests = encoder.transform(y_val)

In [0]:
num_classes = np.max(y_train) + 1
y_trains = utils.to_categorical(y_train, num_classes)
y_tests = utils.to_categorical(y_val, num_classes)

In [0]:
batch_size = 32
epochs = 50


max_features = 10000
# Cut texts after this number of words 
# (among top max_features most common words)
max_len = 100

model = Sequential()
model.add(Dense(512, input_shape=(max_len,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [125]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_val.shape)
print('y_train shape:', y_trains.shape)
print('y_test shape:', y_tests.shape)



x_train shape: (200, 100)
x_test shape: (10000, 100)
y_train shape: (200, 2)
y_test shape: (10000, 2)


In [126]:
history = model.fit(x_train, y_trains,
                    batch_size=32,
                    epochs=15,
                    verbose=1,
                    validation_split=0.1)

Train on 180 samples, validate on 20 samples
Epoch 1/15
180/180 [==============================] - 1s 3ms/step - loss: 7.7009 - acc: 0.5222 - val_loss: 9.6709 - val_acc: 0.4000
Epoch 2/15
180/180 [==============================] - 0s 175us/step - loss: 7.8800 - acc: 0.5111 - val_loss: 10.4768 - val_acc: 0.3500
Epoch 3/15
180/180 [==============================] - 0s 169us/step - loss: 7.2532 - acc: 0.5500 - val_loss: 8.8650 - val_acc: 0.4500
Epoch 4/15
180/180 [==============================] - 0s 168us/step - loss: 7.2274 - acc: 0.5444 - val_loss: 11.2827 - val_acc: 0.3000
Epoch 5/15
180/180 [==============================] - 0s 166us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 11.2827 - val_acc: 0.3000
Epoch 6/15
180/180 [==============================] - 0s 174us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 11.2827 - val_acc: 0.3000
Epoch 7/15
180/180 [==============================] - 0s 166us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 11.2827 - val_acc: 0.3000
Epoch 8/15
180

In [0]:
batch_size = 32
epochs = 50


max_features = 10000
# Cut texts after this number of words 
# (among top max_features most common words)
max_len = 100

model = Sequential()
model.add(Dense(512, input_shape=(max_len,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [128]:
history = model.fit(x_train, y_trains,
                    epochs=10,
                    batch_size=32,
                    verbose=1,
                    validation_split=0.1)




Train on 180 samples, validate on 20 samples
Epoch 1/10
180/180 [==============================] - 1s 4ms/step - loss: 7.1246 - acc: 0.5556 - val_loss: 7.2136 - val_acc: 0.5500
Epoch 2/10
180/180 [==============================] - 0s 160us/step - loss: 7.5275 - acc: 0.5278 - val_loss: 9.6181 - val_acc: 0.4000
Epoch 3/10
180/180 [==============================] - 0s 148us/step - loss: 7.9261 - acc: 0.5056 - val_loss: 10.0257 - val_acc: 0.3500
Epoch 4/10
180/180 [==============================] - 0s 158us/step - loss: 8.5097 - acc: 0.4611 - val_loss: 8.8166 - val_acc: 0.4500
Epoch 5/10
180/180 [==============================] - 0s 169us/step - loss: 7.9192 - acc: 0.5056 - val_loss: 7.2136 - val_acc: 0.5500
Epoch 6/10
180/180 [==============================] - 0s 154us/step - loss: 8.1042 - acc: 0.4944 - val_loss: 8.8166 - val_acc: 0.4500
Epoch 7/10
180/180 [==============================] - 0s 156us/step - loss: 8.2823 - acc: 0.4833 - val_loss: 8.8166 - val_acc: 0.4500
Epoch 8/10
180/180

In [146]:
# import numpy as np
# from matplotlib import pyplot as plt
# import pandas as pnd
# from sklearn.preprocessing import Imputer, LabelEncoder, OneHotEncoder, StandardScaler
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import confusion_matrix

y_pred = model.predict(x_val)





confusion_matrix(
    y_tests.argmax(axis=1), y_pred.argmax(axis=1))


array([[3855, 1101],
       [3856, 1188]])

In [147]:
history = model.fit(x_train, y_trains,
                    epochs=2,
                    batch_size=32,
                    validation_split=0.2)

Train on 160 samples, validate on 40 samples
Epoch 1/2
160/160 [==============================] - 0s 249us/step - loss: 7.4140 - acc: 0.5375 - val_loss: 6.3030 - val_acc: 0.6000
Epoch 2/2
160/160 [==============================] - 0s 182us/step - loss: 8.3109 - acc: 0.4813 - val_loss: 8.4159 - val_acc: 0.4750


In [148]:
y_pred = model.predict(x_val)





confusion_matrix(
    y_tests.argmax(axis=1), y_pred.argmax(axis=1))

array([[3320, 1636],
       [3289, 1755]])

In [0]:
batch_size = 32
epochs = 50


max_features = 10000
# Cut texts after this number of words 
# (among top max_features most common words)
max_len = 100

model = Sequential()
model.add(Dense(512, input_shape=(max_len,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

In [155]:
history = model.fit(x_train, y_trains,
                    epochs=2,
                    batch_size=32,
                    validation_split=0.2)

Train on 160 samples, validate on 40 samples
Epoch 1/2
160/160 [==============================] - 1s 4ms/step - loss: 7.2140 - acc: 0.5500 - val_loss: 6.8129 - val_acc: 0.5750
Epoch 2/2
160/160 [==============================] - 0s 139us/step - loss: 7.9149 - acc: 0.5062 - val_loss: 6.8129 - val_acc: 0.5750


In [156]:
y_pred = model.predict(x_val)





confusion_matrix(
    y_tests.argmax(axis=1), y_pred.argmax(axis=1))

array([[4956,    0],
       [5041,    3]])

In [157]:

from keras.models import model_from_json
import numpy
import os

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
import os
import json
import numpy as np
from random import randrange

In [0]:
def load_data(link):
  
  text = []
  sentiment = []

  for filename in os.listdir(link):
    if filename != '.ipynb_checkpoints':
      file_path = os.path.join(link, filename)
      with open(file_path, encoding = 'unicode_escape') as f:
        dic = json.load(f)

        for k, v in dic['text'].items():
          text.append(v)
        for k, v in dic['sentiment'].items():
          sentiment.append(v)
  return text, sentiment

In [161]:
%cd ../      

/


In [0]:


(text, sentiment)=load_data('home')

In [0]:
def preprocess_data(text, sentiment):
  # 80-20 split
  combo = [list(i) for i in zip(text, sentiment)]
  
  combo_test = []

  test_size = int(len(combo)*0.2)
  while len(combo_test) < test_size:
    index = randrange(len(combo))
    combo_test.append(combo.pop(index))
  
  x_train = []
  y_train = []
  x_test = []
  y_test = []
  # x, y split
  for line in combo:
    x_train.append(line[0])
    if line[1] in ['Positive', 'positive', 'postive']:
      y_train.append(2)
    elif line[1] in ['Neutral', 'neutral', 'neutra', 'neutra;', 'Neural']:
      y_train.append(1)
    elif line[1] in ['Negative', 'negative', 'Negetive']:
      y_train.append(0)
    else:
      print('error in sentiment label {}'.format(line[1]))
  for line in combo_test:
    x_test.append(line[0])
    if line[1] in ['Positive', 'positive', 'postive']:
      y_test.append(2)
    elif line[1] in ['Neutral', 'neutral', 'neutra', 'neutra;', 'Neural']:
      y_test.append(1)
    elif line[1] in ['Negative', 'negative', 'Negetive']:
      y_test.append(0)
    else:
      print('error in sentiment label {}'.format(line[1]))
      
  return (x_train, y_train), (x_test, y_test)

In [0]:
(x_train, y_train), (x_test, y_test) = preprocess_data(text, sentiment)

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils


In [0]:
max_words = 100
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

In [0]:
tokenize.fit_on_texts(x_train) # only fit on train
x_trains = tokenize.texts_to_matrix(x_train)
x_tests = tokenize.texts_to_matrix(x_test)

In [0]:
encoder = LabelEncoder()
encoder.fit(y_train)
y_trains = encoder.transform(y_train)
y_tests = encoder.transform(y_test)

In [0]:
num_classes = np.max(y_train)+1 
y_trains = utils.to_categorical(y_train, num_classes)
y_tests = utils.to_categorical(y_test, num_classes)

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os

In [171]:
cd content

/content


In [172]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")


Loaded model from disk


In [173]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 512)               51712     
_________________________________________________________________
activation_21 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 2)                 1026      
_________________________________________________________________
activation_22 (Activation)   (None, 2)                 0         
Total params: 52,738
Trainable params: 52,738
Non-trainable params: 0
_________________________________________________________________


In [0]:
loaded_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:

model = Sequential()
for layer in loaded_model.layers[:-1]: # just exclude last layer from copying
 model.add(layer)
for layer in model.layers:
 layer.trainable = False
 model.add(Dense(3, activation='softmax'))

In [176]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 512)               51712     
_________________________________________________________________
activation_21 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 2)                 1026      
_________________________________________________________________
dense_29 (Dense)             (None, 3)                 9         
_________________________________________________________________
dense_30 (Dense)             (None, 3)                 12        
_________________________________________________________________
dense_31 (Dense)             (None, 3)                 12        
__________

In [177]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_trains, y_trains,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 1184 samples, validate on 132 samples
Epoch 1/50
1184/1184 [==============================] - 1s 907us/step - loss: 1.3693 - acc: 0.0861 - val_loss: 1.2786 - val_acc: 0.1667
Epoch 2/50
1184/1184 [==============================] - 0s 155us/step - loss: 1.2931 - acc: 0.0861 - val_loss: 1.2230 - val_acc: 0.1667
Epoch 3/50
1184/1184 [==============================] - 0s 159us/step - loss: 1.2272 - acc: 0.1774 - val_loss: 1.1766 - val_acc: 0.3712
Epoch 4/50
1184/1184 [==============================] - 0s 158us/step - loss: 1.1709 - acc: 0.4003 - val_loss: 1.1391 - val_acc: 0.3712
Epoch 5/50
1184/1184 [==============================] - 0s 160us/step - loss: 1.1237 - acc: 0.4003 - val_loss: 1.1083 - val_acc: 0.3712
Epoch 6/50
1184/1184 [==============================] - 0s 163us/step - loss: 1.0844 - acc: 0.4003 - val_loss: 1.0839 - val_acc: 0.3712
Epoch 7/50
1184/1184 [==============================] - 0s 163us/step - loss: 1.0520 - acc: 0.4003 - val_loss: 1.0657 - val_acc: 0.3712
E

In [179]:
y_pred = model.predict(x_tests)





confusion_matrix(
    y_tests.argmax(axis=1), y_pred.argmax(axis=1))

array([[  0,  33,   0],
       [  0, 164,   0],
       [  0, 131,   0]])